In [ ]:
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
from datetime import timedelta


# Valuing options

A financial option is a contract between a buyer and a seller that gives the buyer the right, but not the obligation, to buy or sell an **underlying asset**, such as a stock, at a specific price (**strike price**, $K$) on or before a specific date (**expiration date** or **maturity**, $T$). The seller, or option writer, is obligated to fulfill the terms of the contract if the buyer chooses to exercise their option. Options can be used for a variety of purposes, such as hedging risk or speculating on the price movement of an asset. There are two main types of options: **call** options, which give the buyer the right to buy the underlying asset, and **put** options, which give the buyer the right to sell the underlying asset. For both types of options, one can either **long** them (buy) or **short** them (sell). There are different types of options with different obligations. For now, we will focus on the **European** ones, which can only be executed at the expiration date.

Naturally, the fact that the owner of an option has a right but no obligation means that options have an intrinsic value, as the owner of an option has more freedom than the seller. This means that, as opposed to say, forward contracts, options have a non-zero price (you must *pay* to gain the favourable contractual situation). A major field of finance concerns the valuation of options, that is, to find how the price of the option, $C(t=0)$, is set, and how it changes over time.


As we said, a European call long option gives the owner the right, but not the obligation, to buy the underlying at maturity $T$ and at the strike prince $K$. A rational owner of the option will only exercise it if the price of the underlying, $S(t)$, is larger than the strike price $K$ (otherwise, it is more profitable to throw away the option and directly buy the asset from the market). Thus, there are two regimes: if $S(t)> K$ (*in the money*), the owner can buy the option and sell the asset in the market, making a profit $\pi=S(t)-K-C_0$ (note that we must discount the price of the option, $C_0$, from the payoff). If $S(t)< K$ , the option is worthless, so $\pi(t)=-C_0$ (*out of the money*). This means that options, as opposed to other financial instruments, have a **nonlinear payoff**, which for call long options is:

$$\pi(Y,T) = \max(S(T) - K - C_0, -C_0)$$

The fact that options show non-linear payoffs is the key reason why their pricing is significantly harder than other assets such as stocks and forwards.


# Exercise 1:
1. The above discussion provided the payoff of a long call (European) option at time $T$. Do a similar analysis to get the payoff at time $T$ of:
  - A long put option.
  - A short call option.
  - A short put option.

Plot the four cases as a function of the underlying price $S(T)$, for $K=10$ and $C_0=5$.

**Hint**: *to compute entrywise maxima of two numpy arrays, you must use np.maximum, not np.max*.

2. Plot the payoff at time $T$ of portfolios consisting of:
- A long call and a short put, both with strike $K=10$ and $C_0=5$.
- A long call with strike $K=10$ and a long put with strike $K=20$, both with price $C_0=2$.
Under which financial situation would it be profitable to hold the second portfolio? (e.g. financial recession, high market volatility, strong governement intervention...)

3. Based on your results, explain the intuition behind the *call-put parity relation*: $C(T)-P(T)=S(T)-K$, where $C$ is the payoff of a call option, $P$ is the payoff of the equivalent put option, $S$ is the price of the underlying and $K$ is the strike price.




#The binomial model

The binomial option pricing model is a mathematical model that is used to determine the *fair value* of an option. It is based on the idea that the underlying asset can move to one of two possible states (up or down) at the end of each time period. The model uses a recursive algorithm to calculate the value of the option at each point in time, taking into account the probability of the underlying asset moving up or down and the resulting payoffs from the option.

Recall the basic setup for a one-period binomial model:

- The current price of the underlying is $ S_0$ .
- The risk-free annual interest rate is $ r $ .
- The maturity of the option is $T=N\Delta t$, where $ N $ is the number of binomial steps and $\Delta t$ is the time step length.
- With probability $p$, the price goes up by an amount $u>e^r$:  $S(t+1)=uS(t)$.
- With probability $1-p$, the price goes down by an amount $d< e^r$:  $S(t+1)=dS(t)$.
Note: $d$ does not need to be smaller than one, it is sufficient that it is smaller than $e^r$. This is because an asset that bears risk, yet increases in value less than a risk-free asset is an objectively bad asset (negative Sharpe ratio).  
- For simplicity, we will take $d=1/u$.

Surprisingly, the option price does not depend on the probability *p* of the asset going up or down. Instead, it depends in the so-called *risk-neutral* probability $ q $, given by:
$$
q = \frac{e^{r \Delta t} - d}{u - d}
$$




# Exercise 2


Consider the binomial model for a European call option with only one time step, $N=1$.

* At maturity, the option has one out of two possible values:

$$
C_u = \max(S_0 u - K, 0), \\
C_d = \max(S_0 d - K, 0).
$$
* The value of the option at time $T$ is the **risk-neutral expectation**:
$$
C_0 = \left( q C_u + (1-q) C_d \right),
$$
* Finally, the **net present value (NPV)** of the option is:
$$
C_0 = e^{-r \Delta t} \left( q C_u + (1-q) C_d \right),
$$

1. Briefly explain the mathematical reasoning behind each of the mathemtical steps marked with bullet points (one-two lines per step). Hint: use the lecture notes or slides.
2. Create a function `binomial_call_one_step(S0, K, r, u, d, dt)` that computes the present value of an European long call.
3. Create a general function `binomial_call_multistep(S0, K, r, T, u, d, N)` that extends the model to **N steps**, where  at each time step, the underlying price can go up or down. Run the model to compute and plot the fair value of a European call with strike $K=10$ as a function of the underlying value $S_0$, with the following parameters: risk-free interest rate $r=0.01$, $u=1.2$, $d=1/u$, and $T=10$ years. Plot different $N$: $N=1, 10, 100, 1000$. What do you observe?

**Hint**: *You should first compute all possible values of the underlying at maturity, i.e. $S(T)$ (for instance, if $N=2$, these values would be $u^2S_0$, $udS_0$ and $d^2S_0$). Based on this, compute all possible payoffs of the option at maturity, $C_j(T)$ (the index $i$ labels all possible market movements at maturity). Finally, work backwards, node by node, applying the discounted risk-neutral expectation*:
$$
C_j(t-1)=e^{-r\Delta t}(q C_{j+1}(t)+(1-q) C_j(t))
$$
*(each backward step produces an array with one less element. When reaching $i=0$, you should get an array with a single entry, as you have reached the root of the binomial tree).*

**Hint 2**: *to iterate backwards, you can use a for loop with the iterator `range(N-1,-1,-1)`, which starts at $N-1$ and iterates backwards until 0.*
4. For $N=100$, explore the effects of changing $u, r,$ and $T$. Discuss the economic and financial meaning behind each of the observed changes.

In [ ]:
def binomial_call_one_step(S0,K,r,u,d,dt):



def binomial_call_multistep(S0,K,r,T,u,d,N):



# Pricing real options

We will now use our binomial model to price real options. This type of data can also be downloaded with yfinance. Since the procedure is a bit trickier and we only need it here, I give you the impementation of the code. You just need to run the cell below, and you will get a wonderful DataFrame called call_data, contaning data related to call options from Apple.

In [ ]:
### THIS CELL DOWNLOADS OPTIONS AUTOMATICALLY, DON'T CHANGE THE CODE, JUST RUN THE CELL ###

# Create a Ticker object
ticker_symbol = "AAPL"
stock = yf.Ticker(ticker_symbol)

# Fetch one expiration date for options
expiration_dates = stock.options
expiration_date = expiration_dates[0]  # Adjust the index to select a specific date

# Fetch options data for the chosen expiration date and store them in a df
option_chain = stock.option_chain(expiration_date)
calls_data = option_chain.calls

# Add price of the underlying
underlying_prices = []
for date in calls_data["lastTradeDate"]:
    start_date = date.strftime('%Y-%m-%d')
    end_date = (date + timedelta(days=1)).strftime('%Y-%m-%d')
    close_values = yf.download(ticker_symbol, start=start_date, end=end_date, progress=False)["Close"].values
    underlying_prices.append(close_values[0][0] if len(close_values) > 0 else np.nan)
calls_data["underlyingPrice"] = underlying_prices

# Add expiration date
calls_data['expirationDate'] = pd.to_datetime(expiration_date)

# Ensure "lastTradeDate" is in naive-datetime format
calls_data["lastTradeDate"] = pd.to_datetime(calls_data["lastTradeDate"]).dt.tz_localize(None)
calls_data["lastTradeDate"] = calls_data["lastTradeDate"]

# Visual check that everything went okay
print(calls_data.head())


YF.download() has changed argument auto_adjust default to True
        contractSymbol       lastTradeDate  strike  lastPrice  bid  ask  \
0  AAPL250502C00100000 2025-04-22 17:48:30   100.0      98.70  0.0  0.0   
1  AAPL250502C00110000 2025-04-23 16:07:24   110.0      93.93  0.0  0.0   
2  AAPL250502C00120000 2025-04-23 16:07:24   120.0      83.97  0.0  0.0   
3  AAPL250502C00125000 2025-04-22 18:40:34   125.0      75.17  0.0  0.0   
4  AAPL250502C00130000 2025-04-25 17:38:13   130.0      79.00  0.0  0.0   

   change  percentChange  volume  openInterest  impliedVolatility  inTheMoney  \
0     0.0            0.0     NaN             4            0.00001        True   
1     0.0            0.0     6.0             5            0.00001        True   
2     0.0            0.0     3.0             8            0.00001        True   
3     0.0            0.0    10.0             6            0.00001        True   
4     0.0            0.0     2.0             0            0.00001        True   


We now have a new DataFrame, called calls_data, which contains a ot of real data about Apple call options. From all these columns, we are interested only in the following ones:
- `'lastPrice'`: the call price $C(t)$.
- `'underlyingPrice'`: the underlying price $Y(t)$.
- `'strike'`: the strike price $K$.
- `'expirationDate'`: the time (in datetime format) at which the option may be executed.
- `'lastTradeDate'`: the time that we take as the origin.




# Exercise 3:

Using the real $T$, $S_0$, $K$, and $\sigma$, price the options using the multistep binomial model. Use $dt = 1, N=T//dt$, and $r=0$. For the up and down factors, use $u=\exp(\sigma\sqrt{dt}), \ d = \exp(-\sigma\sqrt{dt})$. Compare your results with the real option prices by plotting one against the other in a scatter plot (draw also the line $y=x$ for reference). Is the binomial model a good predictor of option prices?

**Clue 1:** *for simplicity, you can consider constant volatility of returns (no heteroskedasticity). Compute it from the historical data of the last two years with daily resolution; i.e., use the following time series for the prices:*
`df_prices = yf.download('AAPL', period="2y", interval="1d", progress=False)["Adj Close"]`.

**Clue 2:** *the maturity $T$ is the difference between the expiration time and the last trade time, expressed in days. To convert a datetime object into days, use `my_datetime.dt.days`.*



# The Black–Scholes Model


The **Black-Scholes PDE** describes the evolution of the price of a derivative (e.g., an option) over time under the fopllowing assumptions:

1. Frictionless markets: no transaction costs, taxes...
2. Investors can buy and sell any amount of the underlying asset or option (including fractional shares).

3. The price of the underlying asset follows a geometric Brownian motion.

4. Constant Volatility and Interest Rate.

5. There are no arbitrage opportunities in the market.

6. The derivative can only be exercised at maturity.

7. The underlying asset does not pay dividends.

8. Investors are risk-neutral for pricing purposes.

9. The underlying asset can be traded continuously in time.

**Optional**: Which of these assumptions do you think can undermine the validity of the model?

Under these assumptions, the value of the option at time \( t \) with underlying asset price \( S \) can be shown to follow the following PDE:

$$
\frac{\partial C}{\partial t}
+ \frac{1}{2} \sigma^2 C^2 \frac{\partial^2 C}{\partial S^2}
+ r S \frac{\partial C}{\partial S}
- r C = 0,
$$


For a **European call option**, amazingly, incredibly, miracolously, there exists a closed-form solution:

$$
C(S, t) = S \Phi(d_1) - K e^{-r(T-t)} \Phi(d_2),
$$

where

$$
d_1 = \frac{\ln\left(\frac{S_0}{K}\right) + \left(r + \frac{\sigma^2}{2}\right)T}{\sigma \sqrt{T}},
\quad
d_2 = d_1 \;-\; \sigma \sqrt{T},
$$

and $\Phi(\cdot)$ is the standard normal cumulative distribution function (CDF).

 An intuitive explanation behind the solution of the Back-Scholes equation can be found here: https://www.youtube.com/watch?v=EEM2YBzH-2U


# Exercise 4:


1. Write a function to compute the price of a European call according to the Black-Scholes formula, as a function of the price of the underlying. Plot the results using the following parameters: $K=10$, $r=0.01$, $\sigma = 0.12$, $T=10$.
2. Compare this price to the one predicted by the binomial model for different $N$ and fixed $T=N\Delta t$, starting from $N=1$ (make sure you cover different orders of magnitude of $N$). For the up and down factors, use the relation $u = e^{\sigma \sqrt{\Delta t}}, d = e^{-\sigma \sqrt{\Delta t}}$.


In [ ]:
def black_scholes_option_price(S, K, r, T, sigma):


# Optional: American Options

An **American** option differs from a European option in that it allows the holder to exercise the option **at any time** up to and including maturity, rather than only at maturity. This added flexibility is particularly relevant for **put options**, where early exercise may be optimal if the underlying asset drops significantly in value. As a result, when pricing American options using a binomial tree, you must check at every node before maturity whether exercising immediately (the exercise value) yields a higher payoff than holding the option (the continuation value). The value at the node is the maximum of these two.





# Optional: Exercise 5

**Warning: this exercise is quite advanced.**

Adapt the multi-step binomial model developed before to price an American **put** option. Use the same parameters as for the European call: $K=10$, $r=0.01$, $u=1.2$, $d=1/u$, $T=10$, $N=1, 10, 100, 1000$. Compare your results with the ones for a European call.

To solve the exercise, you must include in the code the fact that American options can be exercised early. Thus, at each node of the binomial tree, the code needs to choose whether it is worth to exercise early. The early exercise will happen only if the expected value of the option in the future is **smaller** than the current value. Mathematically, at each node, we take:
$$ \text{Option Value} = \max \bigl(\text{Future Expected Value}, \max(K - S_{\text{node}}, 0)\bigr)$$



In [ ]:
def american_put_binomial(S0, K, r, u, d, T, N):
